In [166]:
#conda install -c conda-forge implicit==0.4.4

In [167]:
#conda update -n base -c defaults conda

In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [169]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [170]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

Можно сэмплировать товары случайно, но пропорционально какому-либо весу

Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [171]:
items_weights = data.groupby('item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value': 'weight'})
items_weights['weight'] = np.log(items_weights['weight'].map(lambda x: 1 if x < 1 else x))
items_weights['weight'] /= items_weights['weight'].sum()
items_weights['weight'].sum()

0.9999999999999999

In [172]:
items_weights.head(5)

,item_id,weight
0,25671,0.000013
1,26081,0.000000
2,26093,0.000002
3,26190,0.000002
4,26355,0.000003


In [173]:
def weighted_random_recommendation(items_weights, n=5):
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].tolist())
    
    return recs.tolist()

In [174]:
%%time

weighted_random_recommendation(items_weights, n=5)

Wall time: 7 ms


[847207, 13909968, 13672316, 1064060, 989563]

# Задание 2. Улучшение бейзлайнов и ItemItem

Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров

In [175]:
uniq = data_train.groupby('user_id')['item_id'].unique().reset_index()
uniq.rename(columns={'item_id': 'uniq'}, inplace=True)

uniq.head()

,user_id,uniq
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."
2,3,"[866211, 878996, 882830, 904360, 921345, 93194..."
3,4,"[836163, 857849, 877523, 878909, 883932, 89142..."
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69..."


In [176]:
items_weights_5000 = data.groupby('item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value': 'weight'})
items_weights_5000.sort_values('weight', ascending=False, inplace=True)
items_weights_5000['weight'] = np.log(items_weights_5000['weight'].map(lambda x: 1 if x < 1 else x))
items_weights_5000.loc[5000:, 'weights5000'] = 0
items_weights_5000.loc[5000:, 'weights5000'] = 0
items_weights_5000['weights5000'] /= items_weights_5000['weights5000'].sum()
items_weights_5000['weights5000'].sum()

0.0

In [177]:
items_weights_5000 = data.groupby('item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value': 'weight'})
items_weights_5000.sort_values('weight', ascending=False, inplace=True)
items_weights_5000.drop(items_weights_5000.tail(len(items_weights_5000.index) - 5000).index,inplace=True)
items_weights_5000['weight'] = np.log(items_weights_5000['weight'].map(lambda x: 1 if x < 1 else x))
items_weights_5000['weight'] /= items_weights_5000['weight'].sum()
items_weights_5000['weight'].sum()

1.0000000000000002

In [178]:
items_weights_5000.head(5)

,item_id,weight
56233,6534178,0.000410
56193,6533889,0.000335
29195,1029743,0.000332
56228,6534166,0.000325
35054,1082185,0.000321


In [179]:
weighted_random_recommendation(items_weights_5000, n=5)

[941198, 12263024, 1053690, 916990, 962991]

In [180]:
uniq['weight_recommendation'] = uniq['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_5000, n=5))

In [181]:
uniq.head()

,user_id,uniq,weight_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[1131115, 1032512, 1139184, 9396750, 940947]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[946788, 1089024, 1013321, 941797, 957164]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93194...","[1101174, 1071594, 858999, 886370, 7139529]"
3,4,"[836163, 857849, 877523, 878909, 883932, 89142...","[926884, 1096667, 912860, 1089954, 1039589]"
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69...","[859877, 8090521, 1106116, 820165, 1051989]"


In [182]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [183]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [184]:
items = data_train.item_id.unique()
top_5000_items = items[:5000:]

In [185]:
uniq['random_recommendation'] = uniq['user_id'].apply(lambda x: random_recommendation(top_5000_items, n=5))

In [186]:
top_5000_popular = data.groupby('item_id')['sales_value'].sum().reset_index()
top_5000_popular = top_5000_popular.sort_values('sales_value', ascending=False)
top_5000_popular.drop(top_5000_popular.tail(len(top_5000_popular.index) - 5000).index,inplace=True)

In [187]:
uniq['popular_recommendation'] = uniq['user_id'].apply(lambda x: popularity_recommendation(top_5000_popular, n=5))

In [188]:
uniq.head(2)

,user_id,uniq,weight_recommendation,random_recommendation,popular_recommendation
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[1131115, 1032512, 1139184, 9396750, 940947]","[7115539, 1082454, 829515, 1046465, 1083272]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[946788, 1089024, 1013321, 941797, 957164]","[1115874, 959496, 6961492, 1048916, 7146027]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [189]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [190]:
for name_col in uniq.columns[1:]:
    print(f'precision@K = {uniq.apply(lambda row: precision_at_k(row[name_col], row.uniq, 5), axis=1).mean():.3f} for {name_col}')

precision@K = 1.000 for uniq
precision@K = 0.052 for weight_recommendation
precision@K = 0.040 for random_recommendation
precision@K = 0.393 for popular_recommendation


Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾 .

In [191]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [192]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [193]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666

G:\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [194]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)


In [195]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# k1

In [196]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 843 ms


In [197]:
%%time

uniq['itemitem_k1'] = uniq['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

Wall time: 40 ms


# k3

In [198]:
%%time

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 835 ms


In [199]:
%%time

uniq['itemitem_k3'] = uniq['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

Wall time: 44 ms


# k5 and without 6666

In [200]:
%%time

uniq['itemitem_k3_w'] = uniq['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=[itemid_to_id[6666]], 
                                                                recalculate_user=True)
                                                                             ])

Wall time: 54 ms


# k5

In [201]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 866 ms


In [202]:
%%time

uniq['itemitem_k5'] = uniq['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])

Wall time: 53 ms


# k5 and without 6666

In [203]:
%%time

uniq['itemitem_k5_w'] = uniq['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=[itemid_to_id[6666]], 
                                                                recalculate_user=True)
                                                                             ])

Wall time: 65 ms


In [204]:
uniq.head(5)

,user_id,uniq,weight_recommendation,random_recommendation,popular_recommendation,itemitem_k1,itemitem_k3,itemitem_k3_w,itemitem_k5,itemitem_k5_w
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[1131115, 1032512, 1139184, 9396750, 940947]","[7115539, 1082454, 829515, 1046465, 1083272]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 995242, 1029743, 840361]","[6666, 1082185, 981760, 995242, 1029743]","[1082185, 981760, 995242, 1029743, 840361]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1127831, 995242, 1098066]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[946788, 1089024, 1013321, 941797, 957164]","[1115874, 959496, 6961492, 1048916, 7146027]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 840361, 1098066, 826249]","[6666, 1082185, 981760, 840361, 1098066]","[1082185, 981760, 840361, 1098066, 826249]","[6666, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 995242]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93194...","[1101174, 1071594, 858999, 886370, 7139529]","[899184, 949620, 1095538, 873130, 823475]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 1098066, 6534178, 826249]","[6666, 1082185, 981760, 1098066, 6534178]","[1082185, 981760, 1098066, 6534178, 826249]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 995242, 826249]"
3,4,"[836163, 857849, 877523, 878909, 883932, 89142...","[926884, 1096667, 912860, 1089954, 1039589]","[1133378, 1056065, 9395106, 947287, 1057315]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1029743, 840361, 883404, 1044078]","[6666, 1082185, 981760, 1029743, 840361]","[1082185, 981760, 1029743, 840361, 883404]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 995242, 826249]"
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69...","[859877, 8090521, 1106116, 820165, 1051989]","[859010, 1039566, 9677656, 7168978, 895138]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1029743, 961554, 916122, 859075]","[6666, 1082185, 981760, 1029743, 961554]","[1082185, 981760, 1029743, 961554, 916122]","[6666, 1082185, 981760, 995242, 1098066]","[1082185, 981760, 995242, 1098066, 1029743]"


In [205]:
for name_col in uniq.columns[1:]:
    print(f'precision@K = {uniq.apply(lambda row: precision_at_k(row[name_col], row.uniq, 5), axis=1).mean():.3f} for {name_col}')

precision@K = 1.000 for uniq
precision@K = 0.052 for weight_recommendation
precision@K = 0.040 for random_recommendation
precision@K = 0.393 for popular_recommendation
precision@K = 0.795 for itemitem_k1
precision@K = 0.683 for itemitem_k3
precision@K = 0.882 for itemitem_k3_w
precision@K = 0.544 for itemitem_k5
precision@K = 0.647 for itemitem_k5_w
